# MovieLens

먼저 데이터를 불러옵니다.

In [1]:
import pandas as pd

def data_loader(file_path):
    file_list = file_path.split(', ')
    ratings = pd.read_table(file_list[0], sep="::", header=None, names=['UserID', 'MovieID', 'Rating', 'timestamp'])
    users = pd.read_table(file_list[1], sep="::", header=None, names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'])
    movies = pd.read_table(file_list[2], sep="::", header=None, names=['MovieID', 'Title', 'Genres'])
    return ratings, users, movies

file_path = 'ratings.dat, users.dat, movies.dat'
ratings, users, movies = data_loader(file_path)

C:\ProgramData\Anaconda3\envs\tens_2g\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\envs\tens_2g\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\envs\tens_2g\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)
C:\ProgramData\Anaconda3\envs\tens_2g\lib\site-packages\ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """
C:\ProgramData\Anaconda3\envs\tens_2g\lib\site-packages\ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted 

In [52]:
ratings.head()

,UserID,MovieID,Rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [4]:
users.head()

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [5]:
movies.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


추천시스템 디자인을 위해 가장 먼저 해야할 작업은 주어진 데이터에 몇 명의 User와 Item이 있는지 확인하는 것입니다. 

ratings 데이터프레임에서 unique한 user와 item 수를 파악하는 `cnt_unique` 함수를 만듭니다.

In [15]:
def cnt_unique(ratings):
    num_users = ratings['UserID'].nunique()
    num_items = ratings['MovieID'].nunique()
    return num_users, num_items

num_users, num_items = cnt_unique(ratings)
print(num_users)
print(num_items)


6040
3706


In [12]:
ratings['UserID'].unique()

array([   1,    2,    3, ..., 6038, 6039, 6040], dtype=int64)

행이 user, 열이 item이고, 값이 rating인 matrix를 __Rating Matrix__ 라고 합니다.

Rating Matrix의 값이 비어있는 정도를 __Sparsity__ 라고 합니다.

Sparsity가 크면 user와 item 사이의 정보가 부족하여 추천 모델의 성능이 잘 나오기 어렵습니다.

MovieLens 1M의 Sparsity를 측정하는 함수를 만들어 측정해보겠습니다.

In [59]:
def cal_sparsity(ratings):
    """
    전체 행렬의 차원은 user수(6040) X item수(3706) 
    원소의 개수는 6040 * 3706 = 22384240
    UserID로 묶은 다음 user들이 본 영화의 수를 전체 행렬의 원소 개수로 나눈다.
    그 후 sparsity를 소수점 아래 4자리에서 반올림
    """
    rating_matrix = ratings.groupby(['UserID'])['MovieID']
    sparsity = 1 - (rating_matrix.nunique().sum() / (num_users * num_items))
    sparsity = round(sparsity, 4)
    return sparsity

cal_sparsity(ratings)

0.9553